# Análise Comparativa de Modelos

## Bibliotecas

In [25]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

from utils.date import get_season
from utils.evaluation import evaluate_model

## Leitura dos dados

In [27]:
df_raw = pd.read_csv('../data/data.csv')
df = df_raw.copy()

## Pré-processamento

In [ ]:
df['dteday'] = pd.to_datetime(df['dteday'])

# preenchendo dia da semana e mês com base na data
df['weekday'] = df['dteday'].dt.dayofweek
df['mnth'] = df['dteday'].dt.month

# em season, preenche apenas as linhas que possue valores nulos
df.loc[df['season'].isnull(), 'season'] = df.loc[df['season'].isnull(), 'dteday'].apply(get_season)

cols_to_drop = ['instant', 'dteday', 'atemp', 'casual', 'registered']
df.drop(columns=cols_to_drop, inplace=True)
df.dropna(inplace=True)

# transformando variavel alvo
# df['cnt'] = np.log1p(df['cnt'])

# Transformação das variáveis cíclicas usando seno e cosseno para preservar a periodicidade.
# Isso faz com que valores próximos (tipo 23:00 e 00:00) tenham representações numéricas próximas
df['hr_sin'] = np.sin(2 * np.pi * df['hr'] / 24)
df['hr_cos'] = np.cos(2 * np.pi * df['hr'] / 24)
df['mnth_sin'] = np.sin(2 * np.pi * df['mnth'] / 12)
df['mnth_cos'] = np.cos(2 * np.pi * df['mnth'] / 12)

# removendo as colunas originais que agora são redundantes
df.drop(columns=['hr', 'mnth'], inplace=True)

# transformando variáveis categóricas
categorical_features = ['season', 'weekday', 'weathersit']
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

## Divisão dos dados

In [35]:
# Dividindo em treino e teste com base no tempo
df_train = df[df['yr'] == 0]
df_test = df[df['yr'] == 1]

df_train = df_train.drop('yr', axis=1)
df_test = df_test.drop('yr', axis=1)

X_train = df_train.drop('cnt', axis=1)
y_train = df_train['cnt']

X_test = df_test.drop('cnt', axis=1)
y_test = df_test['cnt']

print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de y_train: {y_train.shape}")
print(f"Formato de X_test:  {X_test.shape}")
print(f"Formato de y_test:  {y_test.shape}")

Formato de X_train: (6676, 21)
Formato de y_train: (6676,)
Formato de X_test:  (6779, 21)
Formato de y_test:  (6779,)


In [39]:
tscv = TimeSeriesSplit(n_splits=5)

fold_counter = 1
for train_indices, val_indices in tscv.split(X_train):
    print(f"\nFold {fold_counter}:")
    print(f"  - Dados de Treino:   índices de {train_indices.min()} a {train_indices.max()} (Tamanho: {len(train_indices)})")
    print(f"  - Dados de Validação: índices de {val_indices.min()} a {val_indices.max()} (Tamanho: {len(val_indices)})")
    fold_counter += 1


Fold 1:
  - Dados de Treino:   índices de 0 a 1115 (Tamanho: 1116)
  - Dados de Validação: índices de 1116 a 2227 (Tamanho: 1112)

Fold 2:
  - Dados de Treino:   índices de 0 a 2227 (Tamanho: 2228)
  - Dados de Validação: índices de 2228 a 3339 (Tamanho: 1112)

Fold 3:
  - Dados de Treino:   índices de 0 a 3339 (Tamanho: 3340)
  - Dados de Validação: índices de 3340 a 4451 (Tamanho: 1112)

Fold 4:
  - Dados de Treino:   índices de 0 a 4451 (Tamanho: 4452)
  - Dados de Validação: índices de 4452 a 5563 (Tamanho: 1112)

Fold 5:
  - Dados de Treino:   índices de 0 a 5563 (Tamanho: 5564)
  - Dados de Validação: índices de 5564 a 6675 (Tamanho: 1112)
